In [3]:
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
#from skimage import data
from skimage.filters import gaussian
#from skimage.segmentation import active_contour
import tifffile
import cv2
import random
from skimage.morphology import remove_small_objects, binary_closing, disk, binary_opening
from skimage.segmentation import morphological_geodesic_active_contour, flood_fill
from skimage.filters import sobel
import glob
#from skimage.segmentation import inverse_gaussian_gradient

In [4]:
def sharpen_edges(img):
    blur = (img - cv2.GaussianBlur(img,(5,5),0)) 
    details = img - blur
    return img + details

In [5]:
def get_img(file,frame_number):
    tiff_stack = tifffile.imread(file)  # Load as a stack of images
    num = len(tiff_stack)
    frame_number = frame_number
    img = tiff_stack[frame_number]
    image_normalized = (img - img.min()) / (img.max() - img.min()) * 255
    image = image_normalized.astype(np.uint8)
    return image

In [6]:
def load_tiffs_advanced(base_folder, tif_version, hz_folder, index=None, filename=None, return_paths=False):
    """
    Load TIFF images from a selected version (tif II or tif III) and HZ folder (1HZ, 2HZ, 3HZ, spont).

    Parameters:
    - base_folder (str): Base path where 'tif II' and 'tif III' folders are.
    - tif_version (str): Choose 'tif II' or 'tif III'.
    - hz_folder (str or int): Choose 1, 2, 3 (int) or 'spont' (str).
    - index (int, optional): If given, return only the TIFF at this index.
    - filename (str, optional): If given, load a specific file matching the name (without .tif).
    - return_paths (bool, optional): If True, return file paths instead of loading images.
    
    Returns:
    - list of ndarray or paths (if no index/filename) OR single ndarray/path
    """
    if isinstance(hz_folder, int):
        hz_folder = f"{hz_folder}HZ"
    
    path = os.path.join(base_folder, tif_version, hz_folder)
    
    # Get all TIFF files
    tiff_files = sorted(glob.glob(os.path.join(path, '*.tif')))
    
    if not tiff_files:
        raise FileNotFoundError(f"No TIFF files found in {path}")
    
    if filename is not None:
        matched_files = [f for f in tiff_files if filename in os.path.basename(f)]
        if not matched_files:
            raise FileNotFoundError(f"No file matching {filename} found in {path}")
        if return_paths:
            return matched_files[0]
        else:
            return tifffile.imread(matched_files[0])

    if index is not None:
        if index >= len(tiff_files):
            raise IndexError(f"Index {index} out of range (only {len(tiff_files)} files)")
        if return_paths:
            return tiff_files[index]
        else:
            return tifffile.imread(tiff_files[index])
    
    # If no specific file, load all
    if return_paths:
        return tiff_files
    else:
        images = [tifffile.imread(f) for f in tiff_files]
        return images

In [7]:
def calculate_strain_stress_from_max_area(current_area_px, max_area_px, youngs_modulus_kpa=13, pixel_size_nm=650):
    """
    Calculates strain and stress based on max area and current area.

    :param current_area_px: float, current area in pixels²
    :param max_area_px: float, maximum observed area in pixels²
    :param youngs_modulus_kpa: float, Young’s modulus in kPa
    :param pixel_size_nm: float, pixel size in nanometers
    :return: tuple (real_area_mm², strain, stress_kpa)
    """
    pixel_size_mm = pixel_size_nm / 1e6
    real_area_mm2 = current_area_px * (pixel_size_mm ** 2)
    max_area_mm2 = max_area_px * (pixel_size_mm ** 2)
    
    strain = (max_area_mm2 - real_area_mm2) / max_area_mm2
    stress_kpa = youngs_modulus_kpa * strain
    
    return real_area_mm2, strain, stress_kpa


In [8]:
def preprocess_image(img, display=False, crop_coords=None):
    """
    Preprocess the input image: crop, sharpen, blur, Sobel, threshold, and clean.

    Parameters:
        img (ndarray): Grayscale input image.
        display (bool): If True, display intermediate results.
        crop_coords (tuple): Optional. Crop region as (x_start, x_end, y_start, y_end)

    Returns:
        cleaned (ndarray): Preprocessed binary mask.
    """
    if len(img.shape) == 3:
        img = rgb2gray(img)  # Convert to grayscale

    # Auto-crop to center if crop_coords is not provided
    if crop_coords is None:
        h, w = img.shape
        crop_size = 800
        x_start = max((w - crop_size) // 2, 0)
        y_start = max((h - crop_size) // 2, 0)
        x_end = x_start + crop_size
        y_end = y_start + crop_size
        crop_coords = (x_start, x_end, y_start, y_end)

    # Apply cropping
    x_start, x_end, y_start, y_end = crop_coords
    img = img[y_start:y_end, x_start:x_end]
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(3, 3))
    img = clahe.apply((img * 255).astype(np.uint8)) if img.max() <= 1.0 else clahe.apply(img)


    #img = sharpen_edges(img)

    # Apply Gaussian blur
    img_blur = cv2.GaussianBlur(img, (5, 5), sigmaX=3)

    # Sobel edge detection
    img_sobel = sobel(img_blur)

    # Convert to uint8
    img_sobel_uint8 = (img_sobel * 255).astype(np.uint8)

    # Otsu thresholding
    _, binary = cv2.threshold(img_sobel_uint8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Clean binary mask
    cleaned = remove_small_objects(binary.astype(bool), min_size=90)
    cleaned = binary_closing(cleaned, footprint=disk(18))

    if display:
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(img, cmap='gray')
        plt.title("Cropped Image")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(binary, cmap='gray')
        plt.title("Otsu Threshold on Sobel")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(cleaned, cmap='gray')
        plt.title("Cleaned Binary Mask")
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    return cleaned


In [9]:
def compute_flood_fill_area(cleaned, display=False):
    """
    Apply flood fill from the center and calculate the filled area.

    Parameters:
        cleaned (ndarray): Preprocessed binary mask.
        display (bool): If True, display the result.

    Returns:
        area_px (int): Pixel count of filled region.
    """
    seed_point = (cleaned.shape[0] // 2, cleaned.shape[1] // 2)
    filled_image = flood_fill(cleaned.astype(float), seed_point, new_value=0.5)
    area_px = np.sum(filled_image == 0.5)

    if display:
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(cleaned, cmap='gray')
        plt.title('Original Binary Mask')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(filled_image, cmap='gray')
        plt.title('Flood Filled from Center')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    return area_px

In [10]:
def display_filled_on_image(img, filled_image, fill_value=0.5):
    """
    Overlay the filled region on the original grayscale image.

    Parameters:
        img (ndarray): Original grayscale image.
        filled_image (ndarray): Image with filled region.
        fill_value (float): The value used to identify the filled region.
    """
    filled_mask = filled_image == fill_value

    # Convert grayscale image to RGB
    if img.ndim == 2:
        img_rgb = np.stack([img] * 3, axis=-1)
    else:
        img_rgb = img.copy()

    # Normalize for display if needed
    img_rgb = img_rgb.astype(np.float32)
    if img_rgb.max() > 1:
        img_rgb /= 255.0

    # Overlay red on filled region
    overlay = img_rgb.copy()
    overlay[filled_mask] = [1, 0, 0]  # Red

    # Display
    plt.imshow(overlay)
    plt.title("Flood-Filled Region on Image")
    plt.axis('off')
    plt.show()

In [38]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tifffile

def process_all_tiffs_save_jpgs(base_folder, tif_version, hz_folder, output_folder):
    """
    Process all TIFFs in a given folder, save each graph as JPG and results as Excel.
    If output already exists for a file, skip it.
    """
    tiff_paths = load_tiffs_advanced(base_folder, tif_version, hz_folder, return_paths=True)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for tiff_path in tiff_paths:
        # Get the base filename without extension
        filename = os.path.splitext(os.path.basename(tiff_path))[0]
        
        # Define the output paths for the JPG and Excel files
        excel_path = os.path.join(output_folder, f"{filename}.xlsx")
        jpg_path = os.path.join(output_folder, f"{filename}.jpg")

        # Skip the TIFF if both JPG and Excel already exist
        if os.path.exists(excel_path) and os.path.exists(jpg_path):
            print(f"⚠️ Skipping {filename} (already exists)")
            continue

        print(f"🔄 Processing {filename}...")
        try:
            # Load stack using robust loader
            tiff_stack = tifffile.imread(tiff_path)

            areas_px = []
            for img in tiff_stack:
                cleaned = preprocess_image(img, display=False)
                areas_px.append(compute_flood_fill_area(cleaned, display=False))
            print(areas_px)
            max_area_px = max(area for area in areas_px if area is not None)
            print(max_area_px)
            results = []
            for i, area_px in enumerate(areas_px):
                if area_px is not None:
                    real_area_mm2, strain, stress_kpa = calculate_strain_stress_from_max_area(
                        area_px, max_area_px, youngs_modulus_kpa=13
                    )
                    results.append({
                        "frame": i,
                        "area_px": area_px,
                        "real_area_mm2": real_area_mm2,
                        "strain": strain,
                        "stress_kpa": stress_kpa
                    })
                else:
                    results.append({
                        "frame": i,
                        "area_px": None,
                        "real_area_mm2": None,
                        "strain": None,
                        "stress_kpa": None
                    })

            df = pd.DataFrame(results)

            # Save plot
            plt.figure(figsize=(10, 6))
            plt.plot(df['frame'], df['stress_kpa'], linestyle='-', color='b')
            plt.title(filename, fontsize=14)
            plt.xlabel('Frame Time Point (frames)', fontsize=12)
            plt.ylabel('Contraction Stress (mN/mm^2)', fontsize=12)
            plt.grid(True)
            plt.savefig(jpg_path)
            plt.close()

            # Save Excel
            df.to_excel(excel_path, index=False)

        except Exception as e:
            print(f"❌ Failed to process {filename}: {e}")

    print(f"✅ All files processed and saved in: {output_folder}")


In [12]:
# base_folder = r"\\CaspiLab-Backup\Computers-Backup\CaspiLab-Micro\DiskF\Mirit\SmartHeart snapshot and video troublshooting\trouble 3\Day 7 CF;CM with DMEMF12"
# output_folder = r"C:\Users\CaspiLab\Desktop\smarthreart - Dor's codes\18 TIFFS for comperison\18 TIFFS of 1HZ"
# process_all_tiffs_save_jpgs(base_folder, 'TIFS', 1, output_folder)

In [ ]:
base_folder = r"\\CaspiLab-Backup\Computers-Backup\CaspiLab-Micro\DiskF\Mirit\SmartHeart snapshot and video troublshooting\trouble 3\Day 7 CF;CM with DMEMF12"
output_folder = r"C:\Users\CaspiLab\Desktop\smarthreart - Dor's codes\18 TIFFS for comperison\18 TIFFS of 2HZ"
process_all_tiffs_save_jpgs(base_folder, 'TIFS', 2, output_folder)

⚠️ Skipping SH III_2Hz_B8_002current001 (already exists)
⚠️ Skipping SH III_2Hz_B8_006current001 (already exists)
⚠️ Skipping SH III_2Hz_D8_006current001 (already exists)
⚠️ Skipping SH III_2Hz_E8_003current001 (already exists)
⚠️ Skipping SH III_2Hz_F8_009current001 (already exists)
⚠️ Skipping SH III_2Hz_G8_001current001 (already exists)
⚠️ Skipping SH III_2Hz_G8_003current001 (already exists)
⚠️ Skipping SH III_2Hz_G8_007current001 (already exists)
🔄 Processing SH II_2Hz_A4_001current001...
🔄 Processing SH II_2Hz_A4_002current001...
🔄 Processing SH II_2Hz_A4_003current001...
🔄 Processing SH II_2Hz_A4_004current001...


In [ ]:
base_folder = r"\\CaspiLab-Backup\Computers-Backup\CaspiLab-Micro\DiskF\Mirit\SmartHeart snapshot and video troublshooting\trouble 3\Day 7 CF;CM with DMEMF12"
output_folder = r"C:\Users\CaspiLab\Desktop\smarthreart - Dor's codes\18 TIFFS for comperison\18 TIFFS of 3HZ"
process_all_tiffs_save_jpgs(base_folder, 'TIFS', 3, output_folder)

In [ ]:
base_folder = r"\\CaspiLab-Backup\Computers-Backup\CaspiLab-Micro\DiskF\Mirit\SmartHeart snapshot and video troublshooting\trouble 3\Day 7 CF;CM with DMEMF12"
output_folder = r"C:\Users\CaspiLab\Desktop\smarthreart - Dor's codes\18 TIFFS for comperison\18 TIFFS of spont"
process_all_tiffs_save_jpgs(base_folder, 'TIFS', 'spont', output_folder)